In [14]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sbs
import pandas as pd

# Scaling formulae for LHC octupoles


 Parameter | LHC | FCC-hh |
 --- | --- | --- |
beam energy, $E_0$ [TeV] | 0.45/7 | 3.3/50 |
circumference, $C$ [km] | 27 | 100 |
betatron tune (inj), $Q_x/Q_y$ |  59.28/63.31 | 111.28/109.31 |
betatron tune (top), $Q_x/Q_y$ |  59.31/63.32 | 111.31/109.32 |
synchrotron tune (top), $Q_s$ | $2.2\times10^{-3}$ | $1.2\times10^{-3}$ |
bunch intensity, $N_b$ [ppb]  | $1.15\times10^{11}$ |  $10^{11}$ |
4$\sigma$ rms bunch length (top), $\tau_b$ [ns] | 1.08 | 1.07 |
bunch spacing [ns] | 25 | 25 |
norm. emittance, $\varepsilon_{n}$ [$ \mu m$]  | 2.5  |  2.2 |
transition gamma, $\gamma_{t}$ | 55.7 |  99.33 |
averaged $\beta$-function, $\beta^\text{avg}$ [m] | 72 |  141 |
energy spread (inj), $\Delta E / E_0$ $\times 10^{-3}$ | $0.8$ |   $0.29$ |
energy spread (top), $\Delta E / E_0$ $\times 10^{-3}$ | $0.26$ |   $0.12$ |


|Machine |$L_m$ \[m\]| $\beta^F_x$ \[m\]| $\beta^F_y~$ \[m\]| $\beta^D_x$ \[m\]| $\beta^D_y$ \[m\]| $N_{oct}$ |
| --- | --- | --- | --- | --- | --- | --- | 
|LHC| 0.32 | 175.5 | 36.6 | 30.1 | 178.8 | 2x84 |
|FCC| 0.32 | 343.7 | 71.7 | 60.9 | 350.2 | 2x2100 |

Let's define octupole coefficients such that the resulting tune spread will be: 
$$\Delta Q_x = a_{xx}J_x - a_{xy}J_y$$
$$\Delta Q_y = -a_{yx}J_x + a_{yy}J_y$$

Octupole coefficients are defined as follows: 
$$a_{xx, yy} = \dfrac{3}{8\pi}\sum \beta_{x,y}^2\dfrac{O_3L_m}{B\rho_B}$$,
$$a_{xy, yx} = \dfrac{3}{8\pi}\sum 2\beta_{x}\beta_y\dfrac{O_3L_m}{B\rho_B}$$.
Where $O_3$ is octupole strength, $B\rho_B$ is magnetic rigidity, $L_m$ is length of octupoles

For the LHC octupoles the scaling formulae are: 
$$a_{xx}\epsilon_n = 3.28\dfrac{I_{oct}[A]\epsilon_n[m]}{E^2[TeV]}$$
$$a_{yy}\epsilon_n = 3.43\dfrac{I_{oct}[A]\epsilon_n[m]}{E^2[TeV]}$$
$$a_{xy}\epsilon_n = 2.33\dfrac{I_{oct}[A]\epsilon_n[m]}{E^2[TeV]}$$
$$a_{yx}\epsilon_n = 2.33\dfrac{I_{oct}[A]\epsilon_n[m]}{E^2[TeV]}$$

## LHC to FCC scaling

In [15]:
def coefficients_from_scaling(I_oct, eps_n, Energy):
    coeff = I_oct*eps_n/Energy**2
    a_xx = 3.28*coeff
    a_yy = 3.43*coeff
    a_xy = 2.33*coeff
    a_yx = 2.33*coeff
    return np.array([[a_xx, a_xy],[a_yx, a_yy]])
np.set_printoptions(precision=2)
LHC_octupoles_coefficients = coefficients_from_scaling(550, 2.5e-6, 7)
FCC_octupoles_coefficients = coefficients_from_scaling(550, 2.2e-6, 50)
print(LHC_octupoles_coefficients)
print(FCC_octupoles_coefficients)
print(-1.05e-4, 7.4e-5, 7.4e-5, -1.05e-4) # was in Oliver's code before
print(25*FCC_octupoles_coefficients)

[[9.20e-05 6.54e-05]
 [6.54e-05 9.63e-05]]
[[1.59e-06 1.13e-06]
 [1.13e-06 1.66e-06]]
-0.000105 7.4e-05 7.4e-05 -0.000105
[[3.97e-05 2.82e-05]
 [2.82e-05 4.15e-05]]


|Coefficient | LHC octupoles | FCC octupoles |
|---|---|---|
| $a_{xx}\epsilon_n$ | $9.20 \cdot 10^{-5}$ | $1.59 \cdot 10^{-6}$ |
| $a_{xy}\epsilon_n$ | $6.54 \cdot 10^{-5}$ | $1.13 \cdot 10^{-6}$ |
| $a_{yx}\epsilon_n$ | $6.54 \cdot 10^{-5}$ | $1.66 \cdot 10^{-6}$ |
| $a_{yy}\epsilon_n$ | $9.63 \cdot 10^{-5}$ | $1.13 \cdot 10^{-6}$ |

## PyHEADTAIL octupoles' coefficients

$$J_x = \frac{1}{2}\left( \frac{1 + \alpha_x^2}{\beta_x} x^2 +
2\alpha_x x x' +
\beta_x x'^2\right)$$
$$J_y = \frac{1}{2} \left(\frac{1 + \alpha_y^2}{\beta_y} y^2
            + 2\alpha_y  y y'
            + \beta_y y'^2\right)$$
$$dQ_x = \frac{1}{p_0}(a_{xx}J_x + a_{xy}J_y)$$
$$dQ_y = \frac{1}{p_0}(a_{yy}J_y + a_{yx}J_x)$$


For this definition octupole coefficients will be given by: 
$$ a_{xx} = \frac{3e}{8\pi}(N^F_{oct}\beta_x^F\beta_x^FO_3L_m+N^D_{oct}\beta_x^D\beta_x^DO_3L_m) $$
$$ a_{yy} = \frac{3e}{8\pi}(N^F_{oct}\beta_y^F\beta_y^FO_3L_m+N^D_{oct}\beta_y^D\beta_y^DO_3L_m) $$
$$ a_{xy} = a_{yx} = \frac{3e}{4\pi}(N^F_{oct}\beta_y^F\beta_x^FO_3L_m+N^D_{oct}\beta_y^D\beta_x^DO_3L_m) $$

Octupoles are implemented as a detuner in PyHEADTAIL, I noticed no significant difference between using a single octupole kick vs this detuner implementation.